In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigs
from scipy.integrate import solve_ivp

In [6]:
L = 4 
K = 1 
xspan = np.linspace(-L, L, 81)
epsilon_n = 0
d_epsilon = 0.2
tol = 1e-6
n_max = 5

def harmonic_shoot(x, y, epsilon):
    return [y[1], (K * x**2 - epsilon) * y[0]]

eigenfunction = []
eigenvalue = []

for n in range(n_max):
    epsilon_guess = epsilon_n
    d_epsilon = 0.2
    for _ in range(1000):
        phi0 = [1, np.sqrt(L**2 - epsilon_guess)]
        sol = solve_ivp(harmonic_shoot, [-L, L], phi0, args=(epsilon_guess,), t_eval=xspan)

        phi_guess = sol.y[1, -1] + np.sqrt(L**2 - epsilon_guess) * sol.y[0, -1]
        if abs(phi_guess) < tol:
            eigenvalue.append(epsilon_guess)
            break

        if (-1) ** (n) * phi_guess > 0:
            epsilon_guess += d_epsilon
        else:
            epsilon_guess -= d_epsilon / 2
            d_epsilon /= 2


    norm = np.trapz(sol.y[0] ** 2, xspan)  
    normalized_eigenfunction = abs(sol.y[0]) / np.sqrt(norm)  
    epsilon_n = epsilon_guess + 0.1
    eigenfunction.append(normalized_eigenfunction)

eigenfunctions = np.column_stack(eigenfunction)
eigenvalues = np.array(eigenvalue)

A1 = eigenfunctions
A2 = eigenvalues

In [7]:
L = 4
x = np.arange(-L, L + 0.1, 0.1)
n = len(x)
dx = x[1] - x[0]

H = np.zeros((n - 2, n - 2))

for j in range(n - 2):
    H[j, j] = -2 - (dx**2) * x[j + 1]**2
    if j < n - 3:
        H[j + 1, j] = 1
        H[j, j + 1] = 1

H[0, 0] = H[0, 0] + 4 / 3
H[0, 1] = H[0, 1] - 1 / 3
H[-1, -1] = H [-1, -1] + 4 / 3
H[-1, -2] = H[-1, -2] - 1 / 3

eigvals, eigvecs = eigs(-H, k=5, which='SM')

vals2 = np.vstack([4/3 * eigvecs[0,:] - 1/3 * eigvecs[1, :], eigvecs, 4/3 * eigvecs[-1, :] - 1/3 * eigvecs[-2, :]])

eigenvecs = np.zeros((n, 5))
eigenvals = np.zeros(5)

for j in range (5):
    norm = np.sqrt(np.trapz(vals2[:, j]**2, x))
    eigenvecs[:, j] = np.abs(vals2[:, j] / norm)

eigenvals = np.sort(eigvals[:5] / dx**2)

A3 = eigenvecs
A4 = eigenvals


In [8]:
L = 4
x = np.arange(-L, L + 0.1, 0.1)

def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

hermite = np.array([np.ones_like(x),
    2 * x,
    4 * x**2 - 2,
    8 * x**3 - 12 * x,
    16 * x**4 - 48 * x**2 + 12])


phi = np.zeros((len(x), 5))
for j in range(5):
    phi[:, j] = np.exp(-x**2 / 2) * hermite[j, :] / (np.sqrt(factorial(j) * 2**j * np.sqrt(np.pi)))

ErrorA1 = np.zeros(5)
ErrorA2 = np.zeros(5)
ErrorA3 = np.zeros(5)
ErrorA4 = np.zeros(5)

for j in range(5):
    ErrorA1[j] = np.trapz((np.abs(A1[:, j]) - np.abs(phi[:, j]))**2, x)
    ErrorA2[j] = 100 * abs(A2.real[j] - (2 * (j+1) - 1)) / (2 * (j+1) - 1)
    ErrorA3[j] = np.trapz((np.abs(A3[:, j]) - np.abs(phi[:, j]))**2, x)
    ErrorA4[j] = 100 * abs(A4.real[j] - (2 * (j+1) - 1)) / (2 * (j+1) - 1)

A10 = ErrorA1
A11 = ErrorA2
A12 = ErrorA3
A13 = ErrorA4

print("A10 :\n", A10)
print("A11 :\n", A11)
print("A12 :\n", A12)
print("A13 :\n", A13)


A10 :
 [4.57692773e-08 1.72430976e-07 2.47860414e-07 4.35413370e-07
 1.96490590e-06]
A11 :
 [0.02642089 0.03324661 0.03073889 0.03455707 0.04161944]
A12 :
 [2.33908579e-07 2.39720320e-06 1.81275996e-05 1.53048072e-04
 1.24238600e-03]
A13 :
 [0.0626477  0.10536673 0.1718688  0.28016214 0.54888859]
